In [ ]:
##################
# Load libraries #
##################

from get_strava_data import my_data, process_data, get_elevation

import requests

import pandas as pd
import numpy as np

from PIL import Image
import base64

import ast
import polyline

import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import IFrame
from folium.features import CustomIcon


import plotly.express as px
import plotly.graph_objects as go
from plotly.validators.scatter.marker import SymbolValidator
raw_symbols = SymbolValidator().values
import streamlit as st


In [ ]:
print("folium.__version__",folium.__version__)

In [ ]:
polylines_df = pd.read_csv('./data/processed_data.csv', usecols=['name', 'distance', 'total_elevation_gain', 'average_speed', 'weighted_average_watts', 'suffer_score', 'year', 'month', 'day', 'type', 'map'])
polylines_df = polylines_df[polylines_df.type == 'Ride'] # We'll only use rides which have a map

ride_names = polylines_df.name


# We'll choose rides by index to avoid duplicate names 
ride_index = 3
decoded = pd.json_normalize(polylines_df[polylines_df.index == ride_index]['map'].apply(ast.literal_eval))['summary_polyline'].apply(polyline.decode).values[0]


In [ ]:
decoded

In [ ]:
polylines_df

In [ ]:

polylines_df['decoded_polyline'] = polylines_df['map'].apply(ast.literal_eval)
polylines_df['decoded_polyline'] = pd.json_normalize(polylines_df['decoded_polyline'])['summary_polyline']
# Dropping row with decoded_polyline = None
polylines_df = polylines_df[polylines_df.decoded_polyline.notnull()]

polylines_df['decoded_polyline'] = polylines_df['decoded_polyline'].apply(polyline.decode)

polylines_df.head()

In [ ]:
decoded_1 = polylines_df.decoded_polyline.values[0]
decoded_1

In [ ]:
polylines_df['elevation_profile'] = polylines_df['decoded_polyline'].apply(get_elevation)

In [ ]:
# Adding elevation data from Open Street Map
elevation_profile = [get_elevation(coord[0], coord[1]) for coord in decoded]
elevation_profile_feet = [elevation_profile[i] * 3.28084 for i in range(len(elevation_profile))] # Converting elevation to feet

In [ ]:
# Plotting elevation data
fig, ax = plt.subplots(figsize=(10, 4))
ax = pd.Series(elevation_profile_feet).rolling(3).mean().plot(
    ax=ax, 
    color='red', 
    legend=False
)
ax.set_ylabel('Elevation (ft)')
ax.axes.xaxis.set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
# Saving plot
plt.savefig('./data/elevation_profile.png', dpi=300)

In [ ]:
# Mapping route with folium

png = './data/elevation_profile.png'

encoded = base64.b64encode(open(png, 'rb').read())



html = '<img src="data:image/png;base64,{}">'.format
# iframe = IFrame(html(), width=632+20, height=420+20)
# popup = folium.Popup(iframe, max_width=2650)

centroid = [
    np.mean([coord[0] for coord in decoded]), 
    np.mean([coord[1] for coord in decoded])
]
my_map = folium.Map(location=centroid, zoom_start=13)
folium.PolyLine(decoded).add_to(my_map)

my_map

In [ ]:

lat = [coord[0] for coord in decoded] 
lon = [coord[1] for coord in decoded]


token = MAPBOX_TOKEN = st.secrets['MAPBOX_TOKEN']

fig = go.Figure(go.Scattermapbox(
    mode = "lines",
    lon = lon, lat = lat,
    marker = dict(size = 2, color = "red"),
    line = dict(color = "midnightblue", width = 3),
    # text = '‣',
    textfont=dict(color='#E58606'),
    textposition = 'bottom center',))
fig.update_traces(hovertext='', selector=dict(type='scattermapbox'))
fig.update_layout(
    mapbox = {
        'accesstoken': token,
        'style': "outdoors", 'zoom': 12,
        'center': {'lon': centroid[1], 'lat': centroid[0]}
    },
    margin = {'l': 0, 'r': 0, 't': 0, 'b': 0},
    showlegend = False,
    mapbox_style="light")
    

fig.show()

In [ ]:
name = polylines_df[polylines_df.index == 3]['name'].values[0]
distance = polylines_df[polylines_df.index == 3]['distance'].values[0]
elev_gain = polylines_df[polylines_df.index == 3]['total_elevation_gain'].values[0]
avg_speed = polylines_df[polylines_df.index == 3]['average_speed'].values[0]
avg_power = polylines_df[polylines_df.index == 3]['weighted_average_watts'].values[0] 
suffer = polylines_df[polylines_df.index == 3]['suffer_score'].values[0]

fig = px.line(elevation_profile_feet, x=range(len(elevation_profile_feet)), y=pd.Series(elevation_profile_feet).rolling(5).mean())
fig.update_layout(
        xaxis=dict(
            showline=True,
            showgrid=True,
            showticklabels=False,
            linecolor='rgb(204, 204, 204)',
            linewidth=1,
            ticks='',
            tickfont=dict(
                family='Arial',
                size=12,
                color='rgb(82, 82, 82)',
            ),
        ),
        yaxis=dict(
            showgrid=False,
            zeroline=False,
            showline=False,
            gridcolor = 'rgb(235, 236, 240)',
            showticklabels=True,
            title='Elevation (ft)',
            autorange=False,
            range=[0, 2000],
        ),
        autosize=True,
        hovermode="x unified",
        showlegend=False,
        plot_bgcolor='rgba(0,0,0,0)',
        xaxis_title='',
        margin=dict(l=0, r=0, t=0, b=0),
    )
fig.add_annotation(text=f"<b>RIDE STATS</b>--------------------", 
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    x=0.05,
                    y=0.99,
)
fig.add_annotation(text=f"<b>Name</b>: {name}", 
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    x=0.05,
                    y=0.95,
)                    
fig.add_annotation(text=f"<b>Distance</b>: {distance} miles", 
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    x=0.05,
                    y=0.91,
)
fig.add_annotation(text=f"<b>Elevation Gain</b>: {elev_gain} feet", 
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    x=0.05,
                    y=0.87,
)
fig.add_annotation(text=f"<b>Average Speed</b>: {avg_speed} mph", 
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    x=0.05,
                    y=0.83,
)     
fig.add_annotation(text=f"<b>Weighted Power</b>: {avg_power} Watts", 
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    x=0.05,
                    y=0.79,
)
fig.add_annotation(text=f"<b>Suffer Score</b>: {suffer.astype(int)}", 
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    x=0.05,
                    y=0.75,
)  
fig.add_annotation(text="----------------------------------", 
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    x=0.05,
                    y=0.71,
)


fig.show()

In [ ]:
# TESTING GOOGLE API KEY #

import requests
import streamlit as st
GOOGLE_API_KEY = st.secrets['GOOGLE_API_KEY']



lat = 37.7749
lon = -122.4194


def get_elev_data_GOOGLE(lat, lon):
    url = f"https://maps.googleapis.com/maps/api/elevation/json?locations={lat}%2C{lon}&key={GOOGLE_API_KEY}"

    payload={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)
    response = ast.literal_eval(response.text)
    elevation = response['results'][0]['elevation']
    elevation_feet = elevation * 3.28084

    return elevation_feet

In [ ]:
# Using google API to get elevation data for all points in polyline
elevation_profile_google = [get_elev_data_GOOGLE(coord[0], coord[1]) for coord in decoded]

In [ ]:
elevation_profile_google

In [ ]:
name = polylines_df[polylines_df.index == 3]['name'].values[0]
distance = polylines_df[polylines_df.index == 3]['distance'].values[0]
elev_gain = polylines_df[polylines_df.index == 3]['total_elevation_gain'].values[0]
avg_speed = polylines_df[polylines_df.index == 3]['average_speed'].values[0]
avg_power = polylines_df[polylines_df.index == 3]['weighted_average_watts'].values[0] 
suffer = polylines_df[polylines_df.index == 3]['suffer_score'].values[0]

fig = px.line(elevation_profile_google, x=range(len(elevation_profile_google)), y=pd.Series(elevation_profile_google).rolling(5).mean())
fig.update_layout(
        xaxis=dict(
            showline=True,
            showgrid=True,
            showticklabels=False,
            linecolor='rgb(204, 204, 204)',
            linewidth=1,
            ticks='',
            tickfont=dict(
                family='Arial',
                size=12,
                color='rgb(82, 82, 82)',
            ),
        ),
        yaxis=dict(
            showgrid=False,
            zeroline=False,
            showline=False,
            gridcolor = 'rgb(235, 236, 240)',
            showticklabels=True,
            title='Elevation (ft)',
            autorange=False,
            range=[0, 2000],
        ),
        autosize=True,
        hovermode="x unified",
        showlegend=False,
        plot_bgcolor='rgba(0,0,0,0)',
        xaxis_title='',
        margin=dict(l=0, r=0, t=0, b=0),
    )
fig.add_annotation(text=f"<b>RIDE STATS</b>--------------------", 
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    x=0.05,
                    y=0.99,
)
fig.add_annotation(text=f"<b>Name</b>: {name}", 
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    x=0.05,
                    y=0.95,
)                    
fig.add_annotation(text=f"<b>Distance</b>: {distance} miles", 
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    x=0.05,
                    y=0.91,
)
fig.add_annotation(text=f"<b>Elevation Gain</b>: {elev_gain} feet", 
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    x=0.05,
                    y=0.87,
)
fig.add_annotation(text=f"<b>Average Speed</b>: {avg_speed} mph", 
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    x=0.05,
                    y=0.83,
)     
fig.add_annotation(text=f"<b>Weighted Power</b>: {avg_power} Watts", 
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    x=0.05,
                    y=0.79,
)
fig.add_annotation(text=f"<b>Suffer Score</b>: {suffer.astype(int)}", 
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    x=0.05,
                    y=0.75,
)  
fig.add_annotation(text="----------------------------------", 
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    x=0.05,
                    y=0.71,
)


fig.show()